In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import sys
sys.path.append("../..")
import tensortree
from Bio import SeqIO
import numpy as np
import tensorflow as tf

2024-12-05 10:06:10.744604: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-05 10:06:10.760979: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-05 10:06:10.766053: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-05 10:06:10.778751: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-05 10:06:11.448776: W tensorflow/compiler/tf2

In [2]:
msa_file = "PF00028.fasta"
tree_file = "PF00028.fasta.raxml.bestTree"
alphabet = "ARNDCQEGHILKMFPSTWYV"

In [3]:
# parse fasta
import tensortree.model

leaf_names = []
seqs = []
for record in SeqIO.parse(msa_file, "fasta"):
    leaf_names.append(record.id)
    seqs.append(str(record.seq))

one_hot_leaves = tensortree.util.encode_one_hot(seqs, alphabet=alphabet+".")
# handle gaps
one_hot_leaves = one_hot_leaves[... ,:-1] + one_hot_leaves[..., -1:]/len(alphabet)
one_hot_leaves = one_hot_leaves[:, np.newaxis] # add model dimension

# create tree
tree = tensortree.TreeHandler.read(tree_file)

In [4]:
print("MSA depth:", len(seqs))
print("MSA width:", len(seqs[0]))
print("Tree height:", tree.height)
print("Tree nodes:", tree.num_nodes)

MSA depth: 55
MSA width: 129
Tree height: 12
Tree nodes: 108


In [5]:
# set 10 identical models by copying the branch 
# leaves and rate matrix will be broadcasted
num_models = 10

tree.set_branch_lengths(np.repeat(tree.branch_lengths, num_models, -1))

### TensorFlow

In [6]:
tensortree.set_backend("tensorflow")

In [7]:
R, pi = tensortree.substitution_models.jukes_cantor(d = len(alphabet))
rate_matrix = tensortree.model.backend.make_rate_matrix(R, pi).numpy()

I0000 00:00:1733389572.129748  336592 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1733389572.164842  336592 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1733389572.165088  336592 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1733389572.168046  336592 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [8]:
#wrap in tf function for speed
@tf.function
def loglik():
    return tensortree.model.loglik(one_hot_leaves, leaf_names, tree, rate_matrix, tree.branch_lengths, pi,
                                leaves_are_probabilities=True)

In [9]:
_=loglik() # compile

2024-12-05 10:06:15.004596: I tensorflow/core/util/cuda_solvers.cc:178] Creating GpuSolver handles for stream 0x643a6c4392e0


In [10]:
%%timeit -n 10
    _=loglik()

3.83 ms ± 80 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Pytorch

In [11]:
tensortree.set_backend("pytorch")

In [12]:
import torch 

R, pi = tensortree.substitution_models.jukes_cantor(d = len(alphabet))
rate_matrix = tensortree.model.backend.make_rate_matrix(R, pi).to("cuda")
pi = torch.tensor(pi, device="cuda")
one_hot_leaves_torch = torch.tensor(one_hot_leaves, device="cuda")
tree.set_branch_lengths(torch.tensor(tree.branch_lengths, device="cuda"))

In [13]:
L = tensortree.model.loglik(one_hot_leaves_torch, leaf_names, tree, rate_matrix, tree.branch_lengths, pi,
                            leaves_are_probabilities=True)

In [14]:
%%timeit -n 10
L = tensortree.model.loglik(one_hot_leaves_torch, leaf_names, tree, rate_matrix, tree.branch_lengths, pi,
                            leaves_are_probabilities=True)

9.57 ms ± 56.5 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)
